In [2]:
import neo4j
import pandas as pd
from utils import simplify_to_centroid_if_small
from utils_neo4j import init, station_insert_query, del_stations
from geofox_client import get_geofox_client
from utils_geofox import get_stations, stationdf2rows
import pandas as pd

In [3]:
NEO4J_URI = "bolt://localhost:7689"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "busticket123"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
init(driver)

In [4]:
client = get_geofox_client()

In [5]:
# Fetch the POIs
stations = get_stations(client, loadFromDisk=True)['stations']

In [6]:
df_stations = pd.DataFrame(stations)

In [7]:
df_stations.columns.to_list()

['id',
 'name',
 'city',
 'combinedName',
 'vehicleTypes',
 'coordinate',
 'aliasses',
 'shortcuts']

In [8]:
impcols = ['id', 'name', 'city', 'vehicleTypes', 'coordinate', 'aliasses']

In [9]:
df_stations_impcols = df_stations[impcols]

In [10]:
df_stations_impcols.describe()

,id,name,city,vehicleTypes,coordinate,aliasses
count,8828,8828,8828,8828,8828,2976
unique,8828,7083,1098,32,8828,2976
top,Master:90650901,Schule,Hamburg,[REGIONALBUS],"{'x': 9.536332, 'y': 53.842018}",[Kleve (Itzehoe) Abzw. Rahde]
freq,1,91,2307,5453,1,1


In [12]:
rows = stationdf2rows(df_stations_impcols)

In [14]:
# Execute the query
with driver.session() as session:
    del_stations(session)
    result = session.run(station_insert_query, rows=rows)
    print(f"Imported {result.single()['total']} stations")

driver.close()

Imported 8828 stations
